# Computational Theory Problems

In [82]:
import numpy as np
import hashlib
import os

# Problem 1 — Binary Words and Operations

## Goal
The objective of this problem is to implement the core **bitwise primitives** used by the SHA-256 hashing algorithm. SHA-256 operates on **32-bit words** and relies heavily on logical operations and bit manipulation.

## Description
The following functions were implemented according to the SHA-256 specification:

- **Parity(x, y, z)**  
  Computes the XOR of three 32-bit values:  
  $$x \oplus y \oplus z$$

- **Ch(x, y, z)** *(Choose function)*  
  Selects bits from \(y\) or \(z\) depending on \(x\):  
  $$ (x \land y) \oplus (\neg x \land z) $$

- **Maj(x, y, z)** *(Majority function)*  
  Each output bit is the majority of the corresponding bits of the inputs:  
  $$ (x \land y) \oplus (x \land z) \oplus (y \land z) $$

- **ROTR(x, n)**  
  Performs a circular right rotation of \(x\) by \(n\) bits.

- **SHR(x, n)**  
  Shifts \(x\) to the right by \(n\) bits, filling with zeros.

- **\(\Sigma_0, \Sigma_1\)** *(capital sigma)* and **\(\sigma_0, \sigma_1\)** *(small sigma)*  
  Fixed combinations of rotations and shifts defined by SHA-256.


## Notes
Because Python integers do not naturally overflow, all results are cast to
`np.uint32` to correctly simulate **32-bit modular arithmetic**.

## References

- National Institute of Standards and Technology (NIST),  
  *FIPS PUB 180-4: Secure Hash Standard (SHS)*, 2015.  
  https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf


In [10]:
def Parity(x, y, z):
    """
    Return Parity function of three 32-bit words

    Parity(x, y, z) = x XOR y XOR z
    """
    return np.uint32(x ^ y ^ z)

In [11]:
def Ch(x, y, z):
    """
    Apply Choose (CH) function on three 32-bit words

    Combines inputs using bitwise AND and XOR operations and returns a 32-bit result.
    For each bit position, result is selected from y, if corresponding bit of x is 1, otherwise z is selected
    """
    return np.uint32((x & y) ^ (~x & z))

In [12]:
def Maj(x, y, z):
    """
    Apply Majority (MJ) function to three 32-bit words

    For each bit position, if output bit is 1, two or more corresponding bits of x, y and z are 1
    """
    return np.uint32((x & y) ^ (x & z) ^ (y & z))

In [13]:
def ROTR(x, n, w):
    """
    Circular right rotation

    x is rotated right by n bits within the width of w
    """
    return np.uint32((x >> n) | (x << (w - n )))

In [14]:
def Sigma0(x):
    """
    Sigma0 transformation to 32-bit value

    Rotates input to the right by a fixed number and returns the combined result using bitwise XOR
    """
    x = np.uint32(x)

    return np.uint32(ROTR(x, 2) ^ ROTR(x, 13) ^ ROTR(x, 22))

In [15]:
def Sigma1(x):
    """
    Sigma1 transformation to a 32-bit value

    Rotates input to the right by a fixed number and returns the combined result using bitwise XOR
    """

    x = np.uint32(x)

    return np.uint32(ROTR(x, 6) ^ ROTR(x, 11) ^ ROTR(x, 25))

In [16]:
def SHR(x, n):
    """
    Logical right shift on 32-bit value

    Shifts input right by n bits replaces vacated space with zeros
    """
    return np.uint32(x >> n)

In [17]:
def sigma0(x):
    """
    sigma0 transformation to a 32-bit value

    Rotates input to the right by a fixed number, logical right shift and returns result using bitwise XOR
    """
    x = np.uint32(x)

    return np.uint32(ROTR(x, 7) ^ ROTR(x, 18) ^ SHR(x, 3))

In [18]:
def sigma1(x):
    """
    sigma1 transformation to a 32-bit value

    Rotates input to the right by a fixed number, logical right shift and returns result using bitwise XOR
    """

    x = np.uint32(x)

    return np.uint32(ROTR(x, 17) ^ ROTR(x, 19) ^ SHR(x, 10))

# Problem 2 — Fractional Parts of Cube Roots

## Goal
The goal of this problem is to generate the **SHA-256 round constants** using
the fractional parts of the cube roots of prime numbers.

## Background

SHA-256 defines 64 constants \(K_0 \dots K_{63}\) as:

$$
K_i = \left\lfloor 2^{32} \cdot \text{frac}\left(\sqrt[3]{p_i}\right) \right\rfloor
$$

where \(p_i\) is the *i*-th prime number.

---

## Description

Two main components were implemented:

- **Prime number generation**  
  A function that returns the first \(n\) prime numbers.

- **Fractional cube root extraction**  
  For each prime:
  1. Compute the cube root  
  2. Extract the fractional part  
  3. Multiply by \(2^{32}\)  
  4. Floor the result to obtain a 32-bit constant


## Purpose
Using mathematical constants derived from irrational numbers ensures that the
SHA-256 constants are:
- deterministic
- difficult to manipulate
- publicly verifiable

## References

- National Institute of Standards and Technology (NIST),  
  *FIPS PUB 180-4: Secure Hash Standard (SHS)*, Section 4.2.2, 2015.  
  https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf


In [19]:
def primes(n):
    """
    Generate the first of prime numbers

    Iteratively tests integers for whether its a prime number
    Returns a list containing the first n prime numbers
    """

    #Array to store prime numbers
    primeN = []
    #First number test if its a prime number
    num = 2
    #Until n number of prime numbers have been found
    while len(primeN) < n:
        #Assuming number is prime
        is_prime = True
        #Check if number can be divided by previous prime numbers
        for p in primeN:
            #If p * p is larger than num, stop checking
            if p * p > num:
                break
            #If number divided by p evenly it is not a prime number
            if num % p == 0:
                is_prime = False
                break
        #If number is still prime add to list
        if is_prime:
            primeN.append(num)
        #Move to next number
        num += 1
    #Return list of prime numbers
    return primeN

In [48]:
def fractionalCubeRoots(primeNumbers):
    """
    Calculate 32-bit fractional parts of a cube roots of prime numbers

    extracts fractional part of each cube root, convert to 32bits and prints the value is hexadecimal format
    """
    #Store 32-bit values from fractional roots
    frac32 = []
    #Loop through each prime number
    for prime in primeNumbers:
        #Calculate cube root of prime number
        root = prime ** (1/3)
        #Store fractional part of cube root
        frac = root - int(root)
        #Convert the fractional result to a 32-bit integer
        bits = int(frac * (2**32))
        #Store the 32-bit value
        frac32.append(bits)

    #Print the values in a hexadecimal format
    for i, frac in enumerate(frac32):
        print(f"{frac:08x}", end=" ")
        if (i + 1) % 8 == 0:
            print()
    return frac32

In [3]:
primeNumbers = primes(64)
fractionalCubeRoots(primeNumbers)

428a2f98 71374491 b5c0fbcf e9b5dba5 3956c25b 59f111f1 923f82a4 ab1c5ed5 
d807aa98 12835b01 243185be 550c7dc3 72be5d74 80deb1fe 9bdc06a7 c19bf174 
e49b69c1 efbe4786 0fc19dc6 240ca1cc 2de92c6f 4a7484aa 5cb0a9dc 76f988da 
983e5152 a831c66d b00327c8 bf597fc7 c6e00bf3 d5a79147 06ca6351 14292967 
27b70a85 2e1b2138 4d2c6dfc 53380d13 650a7354 766a0abb 81c2c92e 92722c85 
a2bfe8a1 a81a664b c24b8b70 c76c51a3 d192e819 d6990624 f40e3585 106aa070 
19a4c116 1e376c08 2748774c 34b0bcb5 391c0cb3 4ed8aa4a 5b9cca4f 682e6ff3 
748f82ee 78a5636f 84c87814 8cc70208 90befffa a4506ceb bef9a3f7 c67178f2 


# Problem 3 — Padding

## Goal
The goal of this problem is to implement **SHA-256 message padding** and split
the padded message into **512-bit (64-byte) blocks**.

## Padding Rules
SHA-256 padding follows these steps:

1. Append a single `1` bit (`0x80`)
2. Append `0` bits until the message length is congruent to **448 mod 512**
3. Append the original message length as a **64-bit big-endian integer**

## Description
The function implemented:
- calculates the original message length in bits
- appends padding bytes
- appends the length field
- splits the result into fixed-size 64-byte blocks

## Importance
Padding ensures:
- the message fits SHA-256’s block structure
- the original length is preserved
- collisions due to ambiguous message boundaries are avoided

## References

- National Institute of Standards and Technology (NIST),  
  *FIPS PUB 180-4: Secure Hash Standard (SHS)*, Section 5.1, 2015.  
  https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf

In [4]:
def block_parse(msg: bytes):
    """
    Pad a message and split into a fixed size of blocks

    Apply padding to input and splits into 64-byte blocks
    """

    #Store message length in bits
    msgLenBits = len(msg) * 8
    #Copy message into mutable byte array
    paddedMsg = bytearray(msg)

    #Append a single "1" bit
    paddedMsg.append(0x80)

    #Add 0 bytes until the message length is 56 bytes mod 64
    while (len(paddedMsg) % 64) != (64-8):
            paddedMsg.append(0x00)

    #Append the original message length as a 64-bit big endian integer
    paddedMsg.extend(msgLenBits.to_bytes(8, "big"))

    #Split padded message into 64-byte blocks
    for blockStart in range(0, len(paddedMsg), 64):
        yield bytes(paddedMsg[blockStart:blockStart + 64])

In [5]:
msg = b'a'
for block in block_parse(msg):
    print(block.hex())

61800000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000008


In [76]:
#Implemented from Problem 2 
#SHA-256 round constants from FIPS 180-4 Section 4.2.2
K = np.array(fractionalCubeRoots(primes(64)), dtype=np.uint32)

#Initial SHA-256 has values from FIPS PUB 180-4 5.3.3
H0 = np.array([
    np.uint32(0x6A09E667),
    np.uint32(0xBB67AE85),
    np.uint32(0x3C6EF372),
    np.uint32(0xA54FF53A),
    np.uint32(0x510E527F),
    np.uint32(0x9B05688C),
    np.uint32(0x1F83D9AB),
    np.uint32(0x5BE0CD19),
])

428a2f98 71374491 b5c0fbcf e9b5dba5 3956c25b 59f111f1 923f82a4 ab1c5ed5 
d807aa98 12835b01 243185be 550c7dc3 72be5d74 80deb1fe 9bdc06a7 c19bf174 
e49b69c1 efbe4786 0fc19dc6 240ca1cc 2de92c6f 4a7484aa 5cb0a9dc 76f988da 
983e5152 a831c66d b00327c8 bf597fc7 c6e00bf3 d5a79147 06ca6351 14292967 
27b70a85 2e1b2138 4d2c6dfc 53380d13 650a7354 766a0abb 81c2c92e 92722c85 
a2bfe8a1 a81a664b c24b8b70 c76c51a3 d192e819 d6990624 f40e3585 106aa070 
19a4c116 1e376c08 2748774c 34b0bcb5 391c0cb3 4ed8aa4a 5b9cca4f 682e6ff3 
748f82ee 78a5636f 84c87814 8cc70208 90befffa a4506ceb bef9a3f7 c67178f2 


# Problem 4 — Hashes

## Goal
This problem implements the **SHA-256 hashing algorithm** using the components
developed in previous problems.

## Description
Two main functions were developed:

### Compression Function
- Expands each 512-bit block into 64 words
- Performs 64 rounds of operations using:
  - `Ch`, `Maj`
  - `Σ0`, `Σ1`, `σ0`, `σ1`
  - predefined constants `K[i]`
- Updates the internal hash state

### Final Hash Function
- Pads the input message
- Iterates over all message blocks
- Applies the compression function
- Produces a 256-bit (64 hex characters) digest

## Validation
The implementation was tested using messages of varying lengths to ensure
correct padding and block handling.

## References

- National Institute of Standards and Technology (NIST),  
  *FIPS PUB 180-4: Secure Hash Standard (SHS)*, Sections 6–7, 2015.  
  https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf

In [83]:
def hash(current, block):
    '''
    SHA-256 compression
    Current: 8-word chaning value
    Block: 64-byte (512-bit) message block
    Returns: next chaining value (8 np.uint32 words)
    '''

    #Initial hash values from FIPS PUB 180-4, 5.3.3
    if current is None:
        current = H0.copy()

    #Ensure the chaining value has 8 words
    if len(current) != 8:
        raise ValueError("current must be 8 words")
    #Ensure the message block is exactly 512 bits
    if not isinstance(block, (bytes, bytearray)) or len(block) != 64:
        raise ValueError("block must be 64-bytes")

    #Message schedule array from FIPS PUB 180-4, 6.2.2
    W = [np.uint32(0)] * 64

    #Suppress overflow warning
    with np.errstate(over='ignore'):
        #Parse block into 16 big endian 32-bit words
        for t in range(16):
            W[t] = np.uint32(int.from_bytes(block[4*t:4*t+4], "big"))
            #Extend schedule using small sigma function
        for t in range(16, 64):
            W[t] = np.uint32(
                sigma1(W[t-2]) + W[t-7] + sigma0(W[t-15]) + W[t-16]
            )

        #Working variables a to h start as the current chaining value
        a, b, c, d, e, f, g, h = [np.uint32(x) for x in current]

        #64 round K[t] are SHA-256 round constants, FIPS 180-4, 4.2.2
        for t in range(64):
            T1 = np.uint32(h + Sigma1(e) + Ch(e, f, g) + K[t] + W[t])
            T2 = np.uint32(Sigma0(a) + Maj(a, b, c))

            #Update working variables
            h = g
            g = f
            f = e
            e = np.uint32(d + T1)
            d = c
            c = b
            b = a
            a = np.uint32(T1 + T2)

        #Add the final working variables back into the chaining value
        h0, h1, h2, h3, h4, h5, h6, h7 = [np.uint32(x) for x in current]
        return [
            np.uint32(h0 + a),
            np.uint32(h1 + b),
            np.uint32(h2 + c),
            np.uint32(h3 + d),
            np.uint32(h4 + e),
            np.uint32(h5 + f),
            np.uint32(h6 + g),
            np.uint32(h7 + h),
        ]


In [78]:
def sha256_hash(msg):
    #Chain value
    H = None

    #Split padded message into 512-bit blocks
    blocks = block_parse(msg)
    #Process each block using SHA-256 compression function
    for block in blocks:
        H = hash(H, block)

    #Convert final hash state into 32-byte digest (8 * 32-bit words)
    digest = b""
    for word in H:
        digest += int(word).to_bytes(4, "big")

    #Return final SHA-256 hash
    return digest

In [ ]:
#Test messages of different length
messages = [
    b"",            #Empty message
    b"abc",         #Short message
    b"a" * 20,      #Single block message
    b"a" * 200,     #Multi-block message
]

#Compare my SHA-256 implementations with pythons reference implementation
for msg in messages:
    hash = sha256_hash(msg).hex()
    ref_hash = hashlib.sha256(msg).hexdigest()
    print(f"len = {len(msg)}  match = {hash == ref_hash}")

len = 0  match = True
len = 3  match = True
len = 20  match = True
len = 200  match = True


# Problem 5 — Passwords

## Determining the Passwords

The passwords were recovered by comparing the given SHA-256 hashes against
hashes generated from a known password list (`100k-most-used-passwords-NCSC.txt`). When a generated
hash matched one of the target hashes, the corresponding plaintext password was
identified as correct.

This process revealed the passwords associated with the provided hashes,
demonstrating that the passwords were present in a commonly used wordlist and
were therefore vulnerable to offline attacks.

---

## Method Used

The attack performed was a **dictionary attack**, which proceeds as follows:

1. Load the list of target SHA-256 hashes.
2. Read candidate passwords line-by-line from a wordlist.
3. Remove newline and whitespace characters from each candidate.
4. Compute the SHA-256 hash of each password.
5. Compare the computed hash to the target hashes.
6. Report a match when the hashes are identical.

This method is effective when passwords are weak or commonly used, and when
hashes are stored without additional protections such as salting.

---

## Improving Password Hashing

Several measures can be taken to prevent or significantly weaken the type of
attack used in this problem:

### 1. Salting
A **salt** is a random value added to the password before hashing:
\[
\text{hash} = H(\text{salt} \parallel \text{password})
\]

Salting ensures that identical passwords produce different hashes and prevents
the use of precomputed dictionaries and rainbow tables.

### 2. Key Stretching
Using intentionally slow hash functions increases the computational cost of
each password guess. Algorithms such as:
- bcrypt
- scrypt
- Argon2

perform many iterations internally, making large-scale dictionary attacks
impractical.

### 3. Peppering
A **pepper** is a secret value stored separately from the database and combined
with the password before hashing. Even if the database is compromised, the
hashes remain difficult to crack without the pepper.

### 4. Password Policies
Enforcing strong password requirements (length, complexity, and uniqueness)
reduces the likelihood that passwords appear in common wordlists.

---

## Conclusion

The success of the dictionary attack demonstrates that using unsalted SHA-256
for password storage is insecure. Modern password hashing schemes should employ
salting, key stretching, and secure password policies to protect against
offline password cracking attacks.

## References

- OWASP Cheat Sheet Series,  
  *Password Storage Cheat Sheet*,  
  https://cheatsheetseries.owasp.org/cheatsheets/Password_Storage_Cheat_Sheet.html

- https://github.com/danielmiessler/SecLists/blob/master/Passwords/Common-Credentials/100k-most-used-passwords-NCSC.txt,
  *Commonly used password text file*


In [ ]:
#Target hashes
hashes = [
    "5e884898da28047151d0e56f8dc6292773603d0d6aabbdd62a11ef721d1542d8",
    "873ac9ffea4dd04fa719e8920cd6938f0c23cd678af330939cff53c3d2855f34",
    "b03ddf3ca2e714a6548e7495e2a03f5e824eaac9837cd7f159c67b90fb4b7342"
]

found = 0
#Open password list file
with open('100k-most-used-passwords-NCSC.txt', 'r', encoding='UTF-8') as f:
    #Iterate through each password in the file with line numbers starting at 1
    for line_num, line in enumerate(f, 1):
        #Remove white spaces
        pwd = line.strip()
        #Generate the SHA-256 hash of current passwords
        h = hashlib.sha256(pwd.encode()).hexdigest()
        
        #Check if the generated hash matches any target hashes
        if h in hashes:
            #Output the cracked password and its line number in word list
            print(f"Found at line {line_num:,}: '{pwd}'")
            #Remove matched hash
            hashes.remove(h)
            found += 1
            
            #If all hashes have been cracked stop loop
            if found == len(hashes):
                print("All passwords found")
                break

Found at line 4: 'password'
Found at line 281: 'cheese'
Found at line 1,576: 'P@ssw0rd'
